In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset
file_path = '/content/ObesityDataSet_raw_and_data_sinthetic.csv'
data = pd.read_csv(file_path)

In [4]:
# Encode kolom target kategorikal 'NObeyesdad'
label_encoder = LabelEncoder()  # Inisialisasi LabelEncoder untuk encoding target
data['NObeyesdad'] = label_encoder.fit_transform(data['NObeyesdad'])  # Lakukan encoding pada kolom 'NObeyesdad'

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = data.select_dtypes(include=['object']).columns  # Cari kolom dengan tipe data object (kategorikal)
for kolom in kolom_kategorikal:
    data[kolom] = LabelEncoder().fit_transform(data[kolom])  # Lakukan encoding untuk setiap kolom kategorikal

# Pisahkan fitur dan target
X = data.drop(columns=['NObeyesdad'])  # Pisahkan fitur dengan menghapus kolom 'NObeyesdad'
y = data['NObeyesdad']  # Kolom 'NObeyesdad' sebagai target

# Isi nilai yang hilang dan normalisasi fitur
X = X.fillna(0)  # Isi nilai yang hilang dengan 0
scaler = StandardScaler()  # Inisialisasi objek StandardScaler untuk normalisasi
X = scaler.fit_transform(X)  # Normalisasi fitur

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Pisahkan data menjadi 80% pelatihan dan 20% pengujian

# Konversi data ke tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Konversi data pelatihan ke tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # Konversi data pengujian ke tensor
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Konversi label pelatihan ke tensor
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)  # Konversi label pengujian ke tensor

# Pindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Tentukan perangkat (GPU jika tersedia)
X_train_tensor = X_train_tensor.to(device)  # Pindahkan tensor pelatihan ke perangkat yang dipilih
X_test_tensor = X_test_tensor.to(device)  # Pindahkan tensor pengujian ke perangkat yang dipilih
y_train_tensor = y_train_tensor.to(device)  # Pindahkan label pelatihan ke perangkat yang dipilih
y_test_tensor = y_test_tensor.to(device)  # Pindahkan label pengujian ke perangkat yang dipilih

In [5]:
# Model RNN dengan pooling
class ModelMarkovRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelMarkovRNN, self).__init__()
        # RNN sederhana, batch_first=True agar input berformat (batch_size, sequence_length, input_size)
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True)
        self.jenis_pooling = jenis_pooling  # Jenis pooling yang digunakan: 'max' atau 'avg'
        # Lapisan fully connected untuk klasifikasi (output ukuran_hidden menjadi jumlah_kelas)
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)

    def forward(self, x):
        out, _ = self.rnn(x)  # Forward pass melalui RNN, _ digunakan untuk menyimpan hidden state (tidak digunakan di sini)

        # Jenis pooling berdasarkan parameter jenis_pooling ('max' atau 'avg')
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum, ambil nilai maksimum pada setiap timestep
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata, ambil rata-rata pada setiap timestep

        # Lapisan klasifikasi akhir
        out = self.fc(out)  # Lakukan klasifikasi berdasarkan hasil pooling
        return out  # Kembalikan hasil output

In [6]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Jumlah fitur input (kolom fitur pada data pelatihan)
jumlah_kelas = len(label_encoder.classes_)  # Jumlah kelas output (kategori target yang ter-encode)

# Daftar ukuran hidden layer yang akan diuji untuk RNN
ukuran_hidden_list = [32, 64, 128]  # Ukuran hidden layer yang akan diuji: 32, 64, dan 128 unit

# Daftar metode pooling yang akan diuji: 'max' untuk pooling maksimum, 'avg' untuk pooling rata-rata
jenis_pooling_list = ['max', 'avg']  # Jenis pooling yang akan diuji

# Daftar jumlah epoch yang akan diuji (jumlah iterasi pelatihan untuk tiap eksperimen)
daftar_epoch = [5, 50, 100, 250, 350]  # Uji jumlah epoch yang bervariasi

# Daftar optimizer yang akan diuji: SGD, RMSprop, atau Adam
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']  # Pilihan optimizer untuk eksperimen

In [7]:
# Variabel untuk menyimpan model terbaik setelah eksperimen
model_terbaik = None  # Menyimpan model yang memiliki akurasi terbaik
akurasi_terbaik = 0  # Menyimpan nilai akurasi terbaik yang diperoleh
parameter_terbaik = {}  # Menyimpan parameter hyperparameter terbaik yang digunakan untuk model terbaik
ringkasan_hasil = []  # Daftar untuk menyimpan ringkasan hasil eksperimen hyperparameter (model, akurasi, dll.)

In [8]:
# Eksperimen kombinasi hyperparameter
for ukuran_hidden in ukuran_hidden_list:
    for jenis_pooling in jenis_pooling_list:
        for nama_optimizer in daftar_optimizer:
            for epoch in daftar_epoch:
                print(f"--- Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                # Inisialisasi model dan fungsi loss
                model = ModelMarkovRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)  # Inisialisasi model dengan parameter saat ini
                fungsi_loss = nn.CrossEntropyLoss()  # Menggunakan loss function untuk klasifikasi multi-kelas

                # Pilih optimizer berdasarkan eksperimen yang sedang dilakukan
                if nama_optimizer == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent
                elif nama_optimizer == 'RMSprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)  # Root Mean Square Propagation
                elif nama_optimizer == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adaptive Moment Estimation

                # Pengaturan Early Stopping
                batas_sabar = 10  # Jumlah epoch tanpa perbaikan loss sebelum berhenti
                loss_terbaik = float('inf')  # Inisialisasi nilai loss terbaik
                penghitung_sabar = 0  # Menghitung epoch tanpa peningkatan loss

                # Loop pelatihan
                for ep in range(epoch):
                    model.train()  # Menyalakan mode pelatihan
                    optimizer.zero_grad()  # Bersihkan gradien pada optimizer
                    output = model(X_train_tensor.unsqueeze(1))  # Forward pass pada data pelatihan
                    loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                    loss.backward()  # Backpropagation untuk mengupdate gradien
                    optimizer.step()  # Update parameter berdasarkan gradien

                    # Validasi pada setiap epoch
                    model.eval()  # Menyalakan mode evaluasi untuk validasi
                    with torch.no_grad():  # Nonaktifkan gradien untuk efisiensi memori
                        val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass pada data validasi
                        val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss pada data validasi
                        prediksi_val = val_output.argmax(dim=1)  # Prediksi kelas untuk data validasi
                        akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Hitung akurasi

                    # Log detail pelatihan dan validasi setiap epoch
                    print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                    # Early Stopping untuk menghentikan pelatihan jika validasi tidak membaik
                    if val_loss < loss_terbaik:
                        loss_terbaik = val_loss  # Update loss terbaik
                        penghitung_sabar = 0  # Reset penghitung sabar
                    else:
                        penghitung_sabar += 1  # Increment penghitung sabar

                    # Jika tidak ada perbaikan setelah beberapa epoch, hentikan pelatihan lebih awal
                    if penghitung_sabar >= batas_sabar:
                        print("Early stopping diaktifkan.")
                        break

                # Evaluasi model pada data uji setelah pelatihan selesai
                model.eval()
                with torch.no_grad():
                    prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada data test
                    akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi pada data uji

                # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                print(f"Akurasi Akhir untuk Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                ringkasan_hasil.append(f"{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")  # Simpan hasil eksperimen

                # Simpan model dan parameter terbaik jika akurasi lebih baik
                if akurasi > akurasi_terbaik:
                    akurasi_terbaik = akurasi  # Update akurasi terbaik
                    model_terbaik = model  # Simpan model dengan akurasi terbaik
                    parameter_terbaik = {  # Simpan parameter hyperparameter terbaik
                        'ukuran_hidden': ukuran_hidden,
                        'jenis_pooling': jenis_pooling,
                        'optimizer': nama_optimizer,
                        'epoch': epoch
                    }

Streaming output truncated to the last 5000 lines.
Epoch 253/350, Loss: 1.4512, Val Loss: 1.4527, Akurasi: 49.17%
Epoch 254/350, Loss: 1.4498, Val Loss: 1.4513, Akurasi: 49.17%
Epoch 255/350, Loss: 1.4483, Val Loss: 1.4498, Akurasi: 49.17%
Epoch 256/350, Loss: 1.4469, Val Loss: 1.4484, Akurasi: 48.94%
Epoch 257/350, Loss: 1.4455, Val Loss: 1.4470, Akurasi: 48.94%
Epoch 258/350, Loss: 1.4440, Val Loss: 1.4456, Akurasi: 48.94%
Epoch 259/350, Loss: 1.4426, Val Loss: 1.4442, Akurasi: 48.94%
Epoch 260/350, Loss: 1.4412, Val Loss: 1.4428, Akurasi: 48.94%
Epoch 261/350, Loss: 1.4398, Val Loss: 1.4414, Akurasi: 48.94%
Epoch 262/350, Loss: 1.4384, Val Loss: 1.4400, Akurasi: 49.17%
Epoch 263/350, Loss: 1.4370, Val Loss: 1.4386, Akurasi: 49.17%
Epoch 264/350, Loss: 1.4356, Val Loss: 1.4372, Akurasi: 49.17%
Epoch 265/350, Loss: 1.4342, Val Loss: 1.4358, Akurasi: 49.65%
Epoch 266/350, Loss: 1.4328, Val Loss: 1.4345, Akurasi: 49.65%
Epoch 267/350, Loss: 1.4314, Val Loss: 1.4331, Akurasi: 49.65%
Epoc